In [6]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder, RobustScaler

In [7]:
pwd

'C:\\Users\\635718\\Desktop\\DataScience\\HackerEarth\\FutureDatathon'

In [8]:
## load data
products = pd.read_csv('./Q1ClData/cproducts.csv')
payment = pd.read_csv('./Q1ClData/ctender.csv')

#products = products.head(2000)
#payment = payment.head(2000)

In [9]:
products.head(2)

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
0,BBID_20482,1975-10-20,male,MADHYA PRADESH,453441.0,2016-05-01,2655,BB-INDORE-MALHAR MEGA MALL,2,85246,NONPROMO,NaN,1000446431,PATANJALI CHOCOS 125g,55.0,Payback
1,BBID_20485,1955-07-21,female,NaN,999999.0,2017-03-09,2655,BB-INDORE-MALHAR MEGA MALL,8,78829,NONPROMO,NaN,1000010431,SOO FRESH ROLLS HOT DOG 4P,30.0,Payback


In [10]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11972 entries, 0 to 11971
Data columns (total 16 columns):
customerID                    11972 non-null object
DOB                           11653 non-null object
Gender                        10944 non-null object
State                         10389 non-null object
PinCode                       11155 non-null float64
transactionDate               11972 non-null object
store_code                    11972 non-null int64
store_description             11972 non-null object
till_no                       11972 non-null int64
transaction_number_by_till    11972 non-null int64
promo_code                    11972 non-null object
promotion_description         3462 non-null object
product_code                  11972 non-null int64
product_description           11972 non-null object
sale_price_after_promo        11972 non-null float64
discountUsed                  11972 non-null object
dtypes: float64(2), int64(4), object(10)
memory usage: 1.0+ MB

In [11]:
payment.head(2)

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,tender_type,transaction_number_by_till,payment_amount_by_tender,PaymentUsed
0,BBID_2044,1984-04-02,male,NaN,626125.0,2015-09-03,4986,BB-MADURAI-KOCHADAI VILLAGE,22,GVOWN,9208,1000.00,GiftVoucher
1,BBID_2044,1984-04-02,male,NaN,626125.0,2015-09-03,4986,BB-MADURAI-KOCHADAI VILLAGE,22,CASH,9208,10.46,CASH


In [12]:
payment.shape

(149752, 13)

In [13]:
products.columns

Index(['customerID', 'DOB', 'Gender', 'State', 'PinCode', 'transactionDate',
       'store_code', 'store_description', 'till_no',
       'transaction_number_by_till', 'promo_code', 'promotion_description',
       'product_code', 'product_description', 'sale_price_after_promo',
       'discountUsed'],
      dtype='object')

In [14]:
#Product Missing Data
#Missing Data
total = products.isnull().sum().sort_values(ascending=False)
percent = (products.isnull().sum()/products.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

,Total,Percent
promotion_description,8510,0.710825
State,1583,0.132225
Gender,1028,0.085867
PinCode,817,0.068243
DOB,319,0.026646
discountUsed,0,0.000000


In [15]:
products.nunique()

customerID                    10410
DOB                            7026
Gender                            2
State                            37
PinCode                        1353
transactionDate                 790
store_code                        7
store_description                 7
till_no                          25
transaction_number_by_till    11127
promo_code                     2214
promotion_description          2010
product_code                   6000
product_description            5997
sale_price_after_promo         2945
discountUsed                     28
dtype: int64

In [16]:
products['discountUsed'].value_counts().head(2)

Payback            6683
T24Club,Payback    2085
Name: discountUsed, dtype: int64

In [17]:
products[products['customerID'] == 'BBID_20485']

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
1,BBID_20485,1955-07-21,female,NaN,999999.0,2017-03-09,2655,BB-INDORE-MALHAR MEGA MALL,8,78829,NONPROMO,NaN,1000010431,SOO FRESH ROLLS HOT DOG 4P,30.0,Payback


In [18]:
payment[payment['customerID'] == 'BBID_20485']

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,tender_type,transaction_number_by_till,payment_amount_by_tender,PaymentUsed


In [20]:
drop_columns_products = ['State','PinCode','store_description','till_no','promo_code','promotion_description',
                         'product_description','discountUsed','sale_price_after_promo']
products_cleaned = products.drop(drop_columns_products, axis=1)

In [21]:
payment.columns

Index(['customerID', 'DOB', 'Gender', 'State', 'PinCode', 'transactionDate',
       'store_code', 'store_description', 'till_no', 'tender_type',
       'transaction_number_by_till', 'payment_amount_by_tender',
       'PaymentUsed'],
      dtype='object')

In [22]:
#Payment Missing Data
total = payment.isnull().sum().sort_values(ascending=False)
percent = (payment.isnull().sum()/payment.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

,Total,Percent
State,11650,0.077795
Gender,6663,0.044494
PinCode,5564,0.037155
DOB,4596,0.030691
PaymentUsed,1223,0.008167
payment_amount_by_tender,0,0.000000


In [23]:
payment['PaymentUsed'].nunique()

33

In [25]:
payment.nunique()

customerID                     9380
DOB                            6561
Gender                            2
State                            37
PinCode                        1219
transactionDate                 792
store_code                        7
store_description                 7
till_no                          25
tender_type                      45
transaction_number_by_till    69460
payment_amount_by_tender      74237
PaymentUsed                      33
dtype: int64

In [26]:
payment['tender_type'].unique()

array(['GVOWN', 'CASH', 'EDCAXIS', 'MOBI', 'EDCSBI', 'EDCUTI', 'EDCICICI',
       'EDCUBI', 'PYOC', 'FUTP', 'CR-NOTE(I)', 'EDCHDFC', 'PATM', 'SPCD',
       'MVSODEXHO', 'SFOC', 'CSBK', 'CR-NOTE(R)', '5% Debit Card Disc',
       'GVSUPP', 'Less-VISA 5% Disc', 'CASH-CASHrency', 'PARKCOUP', 'BFBB',
       'EDCAMEX', 'BAJAJ FINANCE', 'CHEQUE', 'CASH-CASHrencyDebitsu',
       'Less-5% Axis Discnt', 'CNIS-Credit Note Issu', 'SOMC', 'MVACCOR',
       'Less -5% Axis Discnt', 'AXISWEDBZR', 'FUTM',
       '5% Debit Card Discnt', 'PCLB', 'FUTP - Oxigen Wallet', 'CCNREDEEM',
       'BHIM', 'FOODCPN', 'Cent CN Redeem', 'FGSC',
       'CASH-CASHrencyDebitar', 'UBCC'], dtype=object)

In [27]:
payment['PaymentUsed'].unique()

array(['GiftVoucher', 'CASH', 'AXIS', 'One MobiKwik Systems',
       'SBI CREDIT CARD', 'UTI', 'ICICI CREDIT CARD', 'UBI CREDIT CARD',
       'Payback', 'FuturePay', 'CREDIT NOTE (ISSUES)', 'HDFC CREDIT CARD',
       'PATM-PAYTM WALLET', 'Store Privilage Card', 'FoodCoupon',
       'SBI Youth Card', 'Cash Back', 'CreditNote', nan, 'SUPPLIER GV',
       'ParkingCoupon', 'BAJAJ FINSERVE', 'AMEX CREDIT CARD', 'CHEQUE',
       'SOMC', 'ACCOR MEAL VOUCHER', '5% Debit Card Discnt',
       'Future Money', 'BBProfitClub', 'CREDIT NOTE (RECEIPTS)',
       'BHIM-AXIS UPI', 'FOOD COUPON', 'FGShoppingFest', 'UBCC'], dtype=object)

In [28]:
payment['PaymentUsed'].value_counts().head(2)

CASH            56838
BBProfitClub    15599
Name: PaymentUsed, dtype: int64

In [29]:
drop_columns_payment = ['DOB','Gender','State','PinCode','store_description','till_no','tender_type','PaymentUsed']
payment_cleaned = payment.drop(drop_columns_payment, axis=1)

In [30]:
products_cleaned.columns

Index(['customerID', 'DOB', 'Gender', 'transactionDate', 'store_code',
       'transaction_number_by_till', 'product_code'],
      dtype='object')

In [31]:
products_cleaned.shape

(11972, 7)

In [33]:
products_cleaned.head(2)

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code
0,BBID_20482,1975-10-20,male,2016-05-01,2655,85246,1000446431
1,BBID_20485,1955-07-21,female,2017-03-09,2655,78829,1000010431


In [34]:
#BBID_20486376
products_cleaned[products_cleaned['customerID'] == 'BBID_20486376']

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code
4681,BBID_20486376,1948-08-09,male,2016-02-05,2655,21038,108001116
4682,BBID_20486376,1948-08-09,male,2016-06-04,2655,97718,300989893
6555,BBID_20486376,1948-08-09,male,2015-10-19,2655,10573,301018402


In [35]:
payment_cleaned[payment_cleaned['customerID'] == 'BBID_20486376']

,customerID,transactionDate,store_code,transaction_number_by_till,payment_amount_by_tender
65319,BBID_20486376,2016-05-04,2655,38847,2083.00
65320,BBID_20486376,2016-06-04,2655,97718,1500.00
65321,BBID_20486376,2016-06-04,2655,97718,1599.94
65322,BBID_20486376,2016-06-04,2655,97718,850.00
65323,BBID_20486376,2016-02-05,2655,21038,4500.00
65324,BBID_20486376,2016-02-05,2655,21038,1297.00
65325,BBID_20486376,2016-02-05,2655,21038,3788.27
80882,BBID_20486376,2015-10-19,2655,10573,2000.00
80883,BBID_20486376,2015-10-19,2655,10573,3641.77
80884,BBID_20486376,2015-10-19,2655,10573,500.00


In [36]:
payment_cleaned.columns

Index(['customerID', 'transactionDate', 'store_code',
       'transaction_number_by_till', 'payment_amount_by_tender'],
      dtype='object')

In [37]:
payment_cleaned.shape

(149752, 5)

In [38]:
payment_cleaned.head(2)

,customerID,transactionDate,store_code,transaction_number_by_till,payment_amount_by_tender
0,BBID_2044,2015-09-03,4986,9208,1000.00
1,BBID_2044,2015-09-03,4986,9208,10.46


In [39]:
#grouping the data in the payment to combine the amount per transaction
payment_cleaned['totalSpend'] = payment_cleaned.groupby(['customerID','transactionDate','store_code',
                                                         'transaction_number_by_till']).transform('sum')

In [40]:
payment_cleaned.head()

,customerID,transactionDate,store_code,transaction_number_by_till,payment_amount_by_tender,totalSpend
0,BBID_2044,2015-09-03,4986,9208,1000.00,1010.46
1,BBID_2044,2015-09-03,4986,9208,10.46,1010.46
2,BBID_2044,2016-04-17,4986,90269,677.28,1677.28
3,BBID_2044,2016-04-17,4986,90268,1042.00,1042.00
4,BBID_2044,2016-04-17,4986,90269,1000.00,1677.28


In [41]:
payment_cleaned.shape

(149752, 6)

In [42]:
payment_cleaned.drop('payment_amount_by_tender', axis=1, inplace=True)

In [43]:
payment_cleaned.shape

(149752, 5)

In [44]:
payment_cleaned.drop_duplicates(inplace=True)

In [45]:
payment_cleaned.shape

(117955, 5)

In [47]:
#lets visualize data
import seaborn as sns
%matplotlib inline

In [48]:
merge_on = ['customerID','transactionDate','store_code','transaction_number_by_till']
merged_data = pd.merge(products_cleaned,payment_cleaned,how='left',on=merge_on)

In [49]:
merged_data.shape

(11972, 8)

In [50]:
merged_data.head()

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code,totalSpend
0,BBID_20482,1975-10-20,male,2016-05-01,2655,85246,1000446431,65.00
1,BBID_20485,1955-07-21,female,2017-03-09,2655,78829,1000010431,NaN
2,BBID_20498,1981-03-19,female,2015-08-12,2655,1064,1000077851005,768.45
3,BBID_20499,1986-05-10,male,2016-12-18,2615,51080,1000443231,1701.24
4,BBID_204110,1988-02-29,male,2015-12-31,2655,21911,300481741,224.89


In [51]:
merged_data['customerID'].value_counts().head(2)

BBID_211419317    7
BBID_20470588     6
Name: customerID, dtype: int64

In [52]:
merged_data[merged_data['customerID'] == 'BBID_20492878']

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code,totalSpend
5367,BBID_20492878,1963-12-03,male,2016-10-23,4986,92253,108000704,5196.86
5368,BBID_20492878,1963-12-03,male,2015-07-08,4986,93578,1000040699005,6037.58
5369,BBID_20492878,1963-12-03,male,2016-07-06,4986,79249,108000666,3833.33
5370,BBID_20492878,1963-12-03,male,2016-07-06,4986,79248,108013313,2619.40


In [59]:
#Merged Data missing values
total = merged_data.isnull().sum().sort_values(ascending=False)
percent = (merged_data.isnull().sum()/merged_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

,Total,Percent
totalSpend,0,0.0
product_code,0,0.0
transaction_number_by_till,0,0.0
store_code,0,0.0
transactionDate,0,0.0
Gender,0,0.0


In [54]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11972 entries, 0 to 11971
Data columns (total 8 columns):
customerID                    11972 non-null object
DOB                           11653 non-null object
Gender                        10944 non-null object
transactionDate               11972 non-null object
store_code                    11972 non-null int64
transaction_number_by_till    11972 non-null int64
product_code                  11972 non-null int64
totalSpend                    10833 non-null float64
dtypes: float64(1), int64(3), object(4)
memory usage: 654.7+ KB


In [55]:
pd.to_datetime("1991-12-03")

Timestamp('1991-12-03 00:00:00')

In [56]:
#add missing data
merged_data['totalSpend'].fillna(merged_data['totalSpend'].mean(), inplace=True)
merged_data['Gender'].fillna('male', inplace=True)
merged_data['DOB'].fillna(pd.to_datetime("1991-12-03"), inplace=True)

In [57]:
#Change Gender to categorical
merged_data['Gender'] = merged_data['Gender'].map( {'female': 0, 'male': 1} ).astype(int)

In [61]:
merged_data[merged_data['DOB'] == 'NANA'] = pd.to_datetime("1991-12-03")

In [62]:
merged_data[merged_data['DOB'] == 'NANA']

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code,totalSpend


In [63]:
#merged_data['MyDOB'] =  pd.to_datetime(merged_data['DOB'], format='%Y-%m-%d')

In [68]:
#get Age from DOB
merged_data['age'] = pd.to_datetime('today').year - pd.to_datetime(merged_data['DOB']).dt.year

In [69]:
#get day of purchase from transactionDate
merged_data['day_of_transaction'] = pd.to_datetime(merged_data['transactionDate']).dt.weekday

In [72]:
#Drop the final set of columns
merged_data = merged_data.drop(['DOB','transactionDate','transaction_number_by_till','MyDOB'], axis=1)

#then start clustering
merged_data.head()

,customerID,Gender,store_code,product_code,totalSpend,age,day_of_transaction
0,BBID_20482,1,2655,1000446431,65,42,6
1,BBID_20485,0,2655,1000010431,3079.66,62,3
2,BBID_20498,0,2655,1000077851005,768.45,36,2
3,BBID_20499,1,2615,1000443231,1701.24,31,6
4,BBID_204110,1,2655,300481741,224.89,29,3


In [75]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11972 entries, 0 to 11971
Data columns (total 7 columns):
customerID            11972 non-null object
Gender                11972 non-null object
store_code            11972 non-null object
product_code          11972 non-null object
totalSpend            11972 non-null object
age                   11972 non-null int64
day_of_transaction    11972 non-null int64
dtypes: int64(2), object(5)
memory usage: 770.4+ KB


In [235]:
## fill missing values

products['promotion_description'].fillna('no_promo', inplace=True)
products['Gender'].fillna('no_gender', inplace=True)
products['State'].fillna('no_state', inplace=True)
products['PinCode'].fillna(-1, inplace=True)
products['DOB'].fillna("1", inplace=True)

In [236]:
## convert data into numeric / float

for c in products.columns:
    lbl = LabelEncoder()
    if products[c].dtype == 'object' and c not in ['store_description','customerID','transactionDate']:
        products[c] = lbl.fit_transform(products[c])

In [237]:
## scaling, creating matrix and running k-means

stores = list(set(products['store_code']))
print(stores)

cluster_labels = []
cluster_store = []
cluster_data = []
cluster_customers = []
cluster_score = []

for x in stores:
    cld = products[products['store_code'] == x]
    cluster_customers.append(cld['customerID'])
    cld.drop(['store_code','customerID','transactionDate','store_description'], axis=1, inplace=True)
    
    print(cld.shape)
    print(cld.head(1))
    
    rbs = RobustScaler()
    cld2 = rbs.fit_transform(cld)
    #print(cld2[0])
    
    km1 = KMeans(n_clusters=3)
    km2 = km1.fit(cld2)
    label = km2.predict(cld2)
    
    s_score = silhouette_score(cld2, label)
    cluster_score.append(s_score)
    #print(s_score)
    
    cluster_labels.append(label)
    cluster_store.append(np.repeat(x, cld.shape[0]))
    cluster_data.append(cld2)

[4843, 3692, 2615, 2906, 4796, 4986, 2655]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


(2135, 12)
       DOB  Gender  State   PinCode  till_no  transaction_number_by_till  \
7071  5870       0     17  560095.0       19                       52921   

      promo_code  promotion_description  product_code  product_description  \
7071        2213                   2010     108009899                 2420   

      sale_price_after_promo  discountUsed  
7071                   219.0            24  
(574, 12)
    DOB  Gender  State   PinCode  till_no  transaction_number_by_till  \
8  3487       0     10  122002.0        5                       11873   

   promo_code  promotion_description  product_code  product_description  \
8        2213                   2010    1000257325                  528   

   sale_price_after_promo  discountUsed  
8                  113.44            24  
(723, 12)
    DOB  Gender  State   PinCode  till_no  transaction_number_by_till  \
3  5224       1     21  400099.0       14                       51080   

   promo_code  promotion_description  pr

In [26]:
# check mean score per store
np.mean(cluster_score)

0.928503444284284

In [27]:
## merge list into ndarray
cluster_data = np.concatenate(cluster_data)

## check if the array has same rows as products file - Yes!
cluster_data.shape

(11972, 12)

In [28]:
## convert nested lists as 1d array
cluster_customers = np.concatenate(cluster_customers)
cluster_store = np.concatenate(cluster_store)
cluster_labels = np.concatenate(cluster_labels)

## create submission files
sub1 = pd.DataFrame({'customerID':cluster_customers, 'store_code':cluster_store, 'cluster':cluster_labels})

np.savetxt('./Q1ClData/subOne_1.txt', cluster_data)
sub1.to_csv('./Q1ClData/subtwo_2.csv', index=False)